In [1]:
!pip3 install gender-guesser
!pip install maven
!pip install sutime
!pip install ethnicolr
!pip install spacy
!python3 -m spacy download en_core_web_sm
!pip install dateparser
!pip install geopy
!pip install allennlp
!pip install allennlp-models

2022-09-21 20:26:31.913704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-21 20:26:31.913767: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-21 20:26:33.979049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-21 20:26:33.980350: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-21 20:26:33.980522: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcub

In [74]:
import json
import re
import numpy as np
import pandas as pd
from os import listdir
import seaborn as sns
from os.path import isfile, join

import gender_guesser.detector as gender

from dateutil.relativedelta import relativedelta
from datetime import datetime

from ethnicolr import pred_wiki_ln, pred_wiki_name
from ethnicolr import census_ln, pred_census_ln
from sutime import SUTime

import spacy
from spacy import displacy 
#nlp = spacy.load('xx_ent_wiki_sm')
nlp = spacy.load('en_core_web_sm')
import dateparser

import geopy
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance

# from allennlp_models import pretrained
# from allennlp.predictors import Predictor
# al = pretrained.load_predictor('tagging-fine-grained-transformer-crf-tagger')

YEAR = 2015
filename = listdir(f"../obit_scraping/all_results/{YEAR}0101-{YEAR}1231")[0]
print(filename)

/home/ajing/.local/lib/python3.10/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


obits-20220423-0229.json


In [9]:
import calendar
from datetime import date

In [10]:
def split_years(start, end):
    d1 = datetime.strptime(start, "%m/%d/%Y").date()
    d2 = datetime.strptime(end, "%m/%d/%Y").date()
    start_year = d1.year
    end_year = d2.year
    safe_start =  date(d1.year+1,1,1)
    safe_end = date(d2.year-1,12,31)
    print(d1.strftime('%m/%d/%Y'), safe_start.strftime('%m/%d/%Y'))
    d = safe_start
    while d < safe_end:
        year_end = date(d.year, 12, 31)
        print(d.strftime('%m/%d/%Y'), year_end.strftime('%m/%d/%Y'))
        d = date(d.year+1,1,1)
    print(safe_end.strftime('%m/%d/%Y'), d2.strftime('%m/%d/%Y'))
split_years("01/08/2020", "10/15/2023")

01/08/2020 01/01/2021
01/01/2021 12/31/2021
01/01/2022 12/31/2022
12/31/2022 10/15/2023


In [11]:
def split_months(start, end):
    d1 = datetime.strptime(start, "%m/%d/%Y").date()
    d2 = datetime.strptime(end, "%m/%d/%Y").date()
    start_year = d1.year
    end_year = d2.year
    safe_start = d1.replace(month=(d1.month+1 if d1.month < 12 else 1), day=1)
    safe_end = d2.replace(month = (d2.month-1 if d2.month > 1 else 12))
    safe_end = safe_end.replace(day=calendar.monthrange(end_year, safe_end.month)[1])
    print(d1.strftime("%m/%d/%Y"), safe_start.strftime("%m/%d/%Y"))
    d = safe_start
    while d < safe_end:
        month_end = d.replace(day=calendar.monthrange(d.year, d.month)[1])
        print(d.strftime("%m/%d/%Y"), month_end.strftime("%m/%d/%Y"))
        d = month_end + relativedelta(days=1)
    print(safe_end.strftime("%m/%d/%Y"), d2.strftime("%m/%d/%Y"))
split_months("01/08/2020", "10/15/2020")


01/08/2020 02/01/2020
02/01/2020 02/29/2020
03/01/2020 03/31/2020
04/01/2020 04/30/2020
05/01/2020 05/31/2020
06/01/2020 06/30/2020
07/01/2020 07/31/2020
08/01/2020 08/31/2020
09/01/2020 09/30/2020
09/30/2020 10/15/2020


In [36]:
def split_weeks(start, end):
    d1 = datetime.strptime(start, "%m/%d/%Y").date()
    d2 = datetime.strptime(end, "%m/%d/%Y").date()
    safe_start = d1 + relativedelta(days = 7 - d1.weekday())
    safe_end = d2 - relativedelta(days = d2.weekday())
    stepsize = relativedelta(days = 7)
    print(d1.strftime("%m/%d/%Y"), safe_start.strftime("%m/%d/%Y"))
    d = safe_start
    while(d < d2):
        week_end = d + relativedelta(days = 6)
        print(d.strftime("%m/%d/%Y"), week_end.strftime("%m/%d/%Y"))
        d = d + relativedelta(days = 7)
    print(safe_end.strftime("%m/%d/%Y"), d2.strftime("%m/%d/%Y"))
split_weeks('01/01/2016', '03/14/2016')

01/01/2016 01/04/2016
01/04/2016 01/10/2016
01/11/2016 01/17/2016
01/18/2016 01/24/2016
01/25/2016 01/31/2016
02/01/2016 02/07/2016
02/08/2016 02/14/2016
02/15/2016 02/21/2016
02/22/2016 02/28/2016
02/29/2016 03/06/2016
03/07/2016 03/13/2016
03/14/2016 03/14/2016


In [75]:
## Open Scrapy results
# obitList = []
# f = open(f"../obit_scraping/all_results/20210101-20211231/{filename}")
# for jsonObj in f:
#     obitDict = json.loads(jsonObj)
#     obitList.append(obitDict)
# df = pd.DataFrame(obitList)

f = open(f"../obit_scraping/all_results/{YEAR}0101-{YEAR}1231/{filename}")
data = json.load(f)
data = [i for i in data if i]
df = pd.DataFrame.from_dict(data)
df = df.drop_duplicates()
f.close()

In [77]:
df

,location,birthdate,deathdate,pubdate,url,para,site_published,name
0,,,,"Jan. 3, 2015",,"ARTHUR MALONEY (Age 85) On Tuesday, December ...",legacy,ARTHUR MALONEY 85
1,,,,"Jan. 12, 2015",,EVERETT D. ABBOTT Members of the Association ...,legacy,Everett D. Abbott
2,,,,"Jan. 9, 2015",,PHILIP S. ADAMS (Age 92) Passed away on Janua...,legacy,PHILIP S. ADAMS
3,"Washington, DC",,,"Jan. 5, 2015",,"STEVEN J. AGRESTA On Friday, January 2, 2015,...",legacy,Steven J. Agresta
4,,,,"Jan. 11, 2015",,"CATHY ANN AKERS On December 29, 2014. She i...",legacy,CATHY ANN AKERS
...,...,...,...,...,...,...,...,...
16529,"Galesville, MD",,,"Dec. 10, 2015",,"HENRY YATES ""Dick"" On December 8, 2015 of Chu...",legacy,HENRY YATES
16530,"Fairfax, VA",,,"Dec. 10, 2015",,"ANTHONY YOUNG ""Tony"" Disability Advocate P...",legacy,ANTHONY YOUNG
16531,"Alexandria, VA",,,"Dec. 31, 2015",,"Stanley T. Young, JR. Stanley T. ""Buddy"" Yo...",legacy,Stanley T. Young Jr.
16532,,,,"Dec. 13, 2015",,"CHARLES DAWSON ZEIGLER On Sunday, November 22...",legacy,CHARLES DAWSON ZEIGLER


### Gender

In [21]:
def guess_gender(df):
    d = gender.Detector()
    def first_name(name):
        ''' Extracts first name 

            Args: 
                name (str): A person's full name
            Returns: 
                The person's first name (str)
        '''
        m = re.match(r'(?:[A-Za-z]{0,2}\. )?([A-Za-z]+) ', name)
        if m:
            return m.group(1).title().strip()
        return ""
    def pronoun_guesser(p):
        mr_count = p.count("Mr")
        ms_count = p.count("Ms")
        mrs_count = p.count("Mrs")
        pronoun_dict = {"unknown": 0,\
                        "male": p.count("Mr")+p.count("he")+p.count("He")+p.count("him")+p.count("Him")+p.count("His")+p.count("his"),\
                        "female": p.count("Ms")+p.count("Mrs")+p.count("she")+p.count("her")+p.count("hers")+p.count("She")+p.count("Her")+p.count("Hers")}
        return max(pronoun_dict, key=pronoun_dict.get)
    df = df.copy()
    df["first_name"] = df["name"].apply(first_name)
    df['sex'] = df.apply(lambda r: d.get_gender(r["first_name"]) if d.get_gender(r["first_name"]) != "unknown" and d.get_gender(r["first_name"]) != "andy" else pronoun_guesser(r["para"]), axis=1)
    
    return df

### Location

In [71]:
def simple_location_finder(df):
    df = df.copy()
    location_finder = re.compile(r'((?:[A-Z][a-z]{2,}\s)?[A-Z][a-z]+,\s[A-Z]{2})')
    def get_loc(p):
        poss_locs = location_finder.findall(p)
        if poss_locs:
            # If the paragraph even mentions 'Washington, DC', throw em in the DC pile!
            if "Washington, DC" in poss_locs or ("DC" in p) or ("Columbia" in p) or ("D.C" in p):
                return "Washington, DC"
            # If the paragraph doesn't mention 'Washington, DC', find the most common location mentioned
            else:
                return max(set(poss_locs), key=poss_locs.count)
        elif ("DC" in p) or ("Columbia" in p) or ("D.C" in p):
            return "Washington, DC"
    df["funeral_home_ind"] = df["funeral_home_name"].apply(lambda x: bool(x))
    first_split = df['funeral_home_address'].str.split('|')
    df['fh_address_line1'] = first_split.str[0]
    second_split = first_split.str[1].str.split(',')
    df['fh_address_city'] = second_split.str[0]
    df['fh_address_state'] = second_split.str[1]
    
    df["newspaper_ind"] = df["newspaper"].apply(lambda x: bool(x))
    df["location"] = df.apply(lambda row: "Washington, DC" if row["location"] == "Washington, DC" else get_loc(row["para"]), axis=1)
    return df

counter = 0
dh = display('0',display_id=True)

def hardcore_location_finder(df):
    global counter
    df = df.copy()
    df["funeral_home_ind"] = df["funeral_home_name"].apply(lambda x: bool(x))
    first_split = df['funeral_home_address'].str.split('|')
    df['fh_address_line1'] = first_split.str[0]
    second_split = first_split.str[1].str.split(',')
    df['fh_address_city'] = second_split.str[0]
    df['fh_address_state'] = second_split.str[1]
    
    df["newspaper_ind"] = df["newspaper"].apply(lambda x: bool(x))
    df_paras = df["para"]
    def location_set(text):
        global counter, dh
        if counter%20 == 0:
            dh.update(f'{round(counter*10000/len(df["para"]))/100}% done')
        locs = []
        d = nlp(text).ents
        for e in d:
            if e.label_ in ['LOC'] or e.label_ in ["GPE"]:
                locs.append(e.text)
        counter += 1
        return locs
    spacy_locs = df_paras.apply(location_set)
    
    locator = geopy.geocoders.Nominatim(user_agent='mygeocoder')
    geocode = RateLimiter(locator.geocode, min_delay_seconds=1, swallow_exceptions=True)
    counter = 0
    def get_dc(p):
        global counter, dh
        if counter%20 == 0:
            dh.update(f'{round(counter*10000/len(spacy_locs))/100}% done')
        counter += 1
        try:
            geo = [geocode(i,addressdetails=True) for i in p]
        except:
            return None
        if geo and len(geo) > 0:
            return list(filter(None, [[i.raw["address"].get("city", None), i.raw["address"].get("state", None)] for i in geo if i]))
        else:
            return None
    
    def combine_address(l):
        if not l:
            return None
        l_states = [i[1] for i in l]
        frequentest = max(set([i for i in l_states if i]), key = l_states.count)
        for i in l:
            if (i[1] == frequentest and i[0]) or i[1] == "District of Columbia":
                return ', '.join(i)
        return frequentest

    sample_addrs = spacy_locs.apply(lambda p: get_dc(p) if len(p) != 0 and p else None)
    df["all_locs"] = sample_addrs
    df["location"] = df["all_locs"].apply(combine_address)
    return df

'0'

In [6]:
# df_wash_num = None
# month_arr = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# df_wash_num = pd.DataFrame(columns = ["month",'num_dc_{}'.format(YEAR)])
# df_wash_num["month"] = month_arr
# month_data = []
# for m in month_arr:
#     month_data.append(len(wash_obits[wash_obits['pubdate'].str[:3] == m]))
# df_wash_num['num_dc_{}'.format(YEAR)] = month_data
# display(df_wash_num)
# df_wash_num.to_csv(f'DC_obits_per_month_{YEAR}.csv', index=False)

### Date finder

In [23]:
## Get correct date formats from SUTime return object (called in get_all_dates(...))
def transform_to_dtformat(date_list):

    ret_list = []
    for elem in date_list:
        if elem['type'] == 'DATE':
            ret_list.append(elem['value'])
        ## Date range
        elif (elem['type'] == 'DURATION') and (isinstance(elem['value'], dict)) and (len(elem['value']) != 0):
            try:
                begin = elem['value']['begin']
            except:
                continue
            if (begin[:2] == 'XX'):
                century = '19' if (begin[2:4] > str(YEAR - 2000)) else '20'
                begin = century + begin[2:]
            try:
                end = elem['value']['end']
            except:
                continue
            if (end[:2] == 'XX'):
                century = '19' if (end[2:4] > str(YEAR - 2000)) else '20'
                end = century + end[2:]
            ret_list.append(begin)
            ret_list.append(end)
        else:
            pass
    
    return ret_list
    
## Creates new column with list of dates (string) obtained from SUTime
def get_all_dates_SU(df):
    sutime = SUTime(mark_time_ranges=True, include_range=True)
    df['all_dates_su'] = df['para'].apply(lambda p: sutime.parse(p))
    df['all_dates_su'] = df['all_dates_su'].apply(lambda res: transform_to_dtformat(res))
    return df
    

## Creates new column with list of dates (string) obtained from SpaCy
def get_all_dates_SP(df):
      
    def get_dates(text):
        dates = set()
        d = nlp(text).ents
        for e in d:
            if e.label_ in ['DATE']:
                dates.add(e.text)
        return list(dates)
    
    df['all_dates_sp'] = df['para'].apply(lambda p: get_dates(p))
    return df


## Creates new columns with 1) list of dates, and 2) Age, obtained from our own regex expressions
def get_all_dates_RE(df):
    date_finder = re.compile(r'(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?) (?:([0-9]{1,2})(?:rd|st|th)?)(?:, ([0-9]{4}))?')
    age_finder = re.compile(r'[Aa]ge ([0-9]{1,3})')
    yrsold_finder = re.compile(r'([0-9]{1,3}) [Yy]ears [Oo]ld')
    
    df["all_dates_re"] = df["para"].str.findall(date_finder) ## Dates found by regex
    
    ## Ages found by regex
    df["para_ages"] = df["para"].str.findall(age_finder) 
    df['para_ages'] = df['para_ages'] + df['para'].str.findall(yrsold_finder)
    df["age"] = df["para_ages"].apply(lambda x: int(max(x)) if len(x)>0 else None)
    
    return df

In [10]:
# from allennlp.predictors import Predictor


# ## Age finder using allennlp package
# def get_age_nlp(para):
#     pred = al.predict(para)
#     ## Finding all numbers from the text
#     date_index = [i for i in range(len(pred['tags'])) if 'DATE' in pred['tags'][i]]
#     date_list = [pred['words'][i] for i in date_index]
#     ## Maximum number that is less than 120 is assigned as age
#     temp_max = 0
#     for num in [int(n) for n in date_list if n.isdigit()]:
#         if (num < 120 and num > temp_max):
#             temp_max = num

#     return temp_max if temp_max != 0 else None

In [24]:
## Tries to parse date (helper function)
def try_parsing_date(text):
    ret_date = dateparser.parse(text, settings={'PREFER_DAY_OF_MONTH': 'first', 'PREFER_DATES_FROM': 'past', 'RELATIVE_BASE': datetime(YEAR, 1, 1)})
    if (not pd.isnull(ret_date)):
        ret_date = ret_date.replace(tzinfo = None)
    if ((not pd.isnull(ret_date)) and (ret_date > datetime.fromisoformat('1900-01-01')) and (ret_date < datetime.today())):
        return ret_date
    else:
        return None


## Creates new columns for birthday and deathday containing 1) datetime object, 2) string and 3) parser used 
## birthday: earliest date
## deathday: most-current date before publish date
def parse_datestrings(df, year, method): 
    
    for i in df.index:
        
        date_objs = {}
        pub_date = try_parsing_date(df.loc[i, 'pubdate']) ## Try get publish date into datetime format
        if (pd.isnull(pub_date)):
            pub_date = try_parsing_date('{}-12-31'.format(year)) ## Defaults to last day of the year
            
        dates = df.loc[i, 'all_dates'] ## Dates for this row
        
        for d in dates:
            ## Date extracted using Regex
            if isinstance(d, tuple):
                date_year = d[2]
                if date_year == '':
                    date_year = year
                
                try:
                    parsed_date = datetime(int(date_year), datetime.strptime(d[0][:3], '%b').month, int(d[1]))
                except:
                    ## Rare case observed: Feb 29th in non-leap year. Resolve by throwing away the day.
                    #parsed_date = datetime(int(date_year), datetime.strptime(d[0][:3], '%b').month)
                    parsed_date = None

                if parsed_date is not None:
                    string = ' '.join(d)
                    method_i = 'regex'

                    date_objs[string] = [parsed_date, method_i]

            ## Date extracted using non-regex (spaCy or SUTime)
            else:
                parsed_date = try_parsing_date(d)
                if parsed_date is not None:
                    string = d
                    method_i = method

                    date_objs[string] = [parsed_date, method_i]
                    
        birthday, deathday = get_birth_death(date_objs, pub_date, year)
        
        if (not pd.isnull(birthday)):
            df.loc[i, 'birthday'] = birthday[0]
            df.loc[i, 'bday_string'] = birthday[1]
            df.loc[i, 'bday_method'] = birthday[2]
        else:
            df.loc[i, 'birthday'] = np.nan
        
        if (not pd.isnull(deathday)):
            df.loc[i, 'deathday'] = deathday[0]
            df.loc[i, 'dday_string'] = deathday[1]
            df.loc[i, 'dday_method'] = deathday[2]
        else:
            df.loc[i, 'deathday'] = np.nan
    
    return df

## Returns birthday and deathday objects from a dictionary of dates passed in
## Input: date_objs is a dictionary with format {'datestring': [datetime_object, parser]}
## Output: birthday and deathdays are tuples with format (datetime_obj, datestring, parser)
def get_birth_death(date_objs, pub_date, year):
    birthday = None
    deathday = None
    
    if len(date_objs) > 0:
        
        ## Bday conditions: more than 20 years ago, and after 1900
        possible_birthdates = {}
        for (key, value) in date_objs.items():
            date = value[0]
            if (date.year < year-20 and date.year > 1900):
                possible_birthdates[key] = value
        
        ## Birthday is set to the tuple associated with MIN datetime object.     
        if len(possible_birthdates) > 0:
            birthday = min([(possible_birthdates[key][0],key, possible_birthdates[key][1]) for key in possible_birthdates], key=lambda t: t[0])
        
        ## Dday conditions: after 1950 and less than publish date
        possible_deathdates = {}
        for (key, value) in date_objs.items():
            date = value[0]
            if (date.year > 1950 and date < pub_date):
                possible_deathdates[key] = value
        
        ## Deathdate is set to tuple associated with MAX datetime object        
        if len(possible_deathdates) > 0:
            deathday = max([(possible_deathdates[key][0], key, possible_deathdates[key][1]) for key in possible_deathdates], key=lambda t: t[0])

    return (birthday, deathday)



In [25]:
## Populate missing or incorrect 'birthday' and 'deathday' fields using scraped dates
def add_scraped_dates(df):

    for i in df.index:
        
        ## Birthdays and deathdays found from the paragraph.
        birthday_para = df.loc[i, 'birthday']
        try: 
            deathday_para = df.loc[i, 'deathday']
        except KeyError:
            print(df.loc[i,:])
        ## Birthdays and deathdays scraped from page HTML
        birthday_scraped = try_parsing_date(df.loc[i, 'birthdate'])
        deathday_scraped = try_parsing_date(df.loc[i, 'deathdate'])
        
        ## If we are missing birthday from paragraph OR if birthday year from paragraph does not match scraped birthday year
        if (not pd.isnull(birthday_scraped) and
            (pd.isnull(birthday_para) or (birthday_para.year != birthday_scraped.year))):
            df.loc[i, 'birthday'] = birthday_scraped ## Set 'birthday' as scraped birthdate
            df.loc[i, 'bday_string'] = df.loc[i, 'birthdate'] ## Original string for scraped birthday
            df.loc[i, 'bday_method'] = 'scraped'
            
        if (not pd.isnull(deathday_scraped) and
            (pd.isnull(deathday_para) or (deathday_para.year != deathday_scraped.year))):
            df.loc[i, 'deathday'] = deathday_scraped
            df.loc[i, 'dday_string'] = df.loc[i, 'deathdate']
            df.loc[i, 'dday_method'] = 'scraped'
                
    return df

## Deduce missing birthdays from age and death date when available.
## Assumes 'birthday' and 'deathday' fields already populated and checked with add_scraped_dates(..).
## Note: Order of reliabily of birthdays: 1) Scraped dates, 2) Death - Age, 3) Paragraph birthday
def birthday_from_age(df):
    for i in df.index:
        birthday = df.loc[i, 'birthday']
        scraped_birthday = try_parsing_date(df.loc[i, 'birthdate'])
        deathday = df.loc[i, 'deathday']
        age = df.loc[i, 'age']
        if ((not pd.isnull(age)) and (not pd.isnull(deathday)) and age < 120):
            deduced_bday = deathday - relativedelta(years = age)
            ## If birthday field does not match (Death - Age), and there is no scraped birthday, replace birthday field with 
            ## min(Death-Age, paragraph birthday)
            if (pd.isnull(scraped_birthday) and (pd.isnull(birthday) or (deduced_bday.year < birthday.year))):
                df.loc[i, 'birthday'] = deduced_bday
                df.loc[i, 'bday_string'] = np.nan
                df.loc[i, 'bday_method'] = 'deduced (death-age)'
    return df

In [26]:
## Run entire workflow for getting dates
## Input: df (dataframe) is raw scraped data, parser (string) is either 'spacy' or 'sutime'
def run_dates(df, parser):
    ret_df = df.copy()
    
    ret_df = get_all_dates_RE(ret_df)
    # ret_df['age_nlp'] = ret_df['para'].apply(lambda x: get_age_nlp(x))
    # ret_df['age'] = ret_df['age'].fillna(ret_df['age_nlp'])

    if (parser == 'spacy'):
        ret_df = get_all_dates_SP(ret_df)
        ret_df['all_dates'] = ret_df['all_dates_sp'] + ret_df['all_dates_re']
    elif (parser == 'sutime'):
        ret_df = get_all_dates_SU(ret_df)
        ret_df['all_dates'] = ret_df['all_dates_su'] + ret_df['all_dates_re']
    else:
        print('invalid parser: must be spacy or sutime')

    ret_df = parse_datestrings(ret_df, YEAR, parser)

    ret_df = add_scraped_dates(ret_df)
    ret_df = birthday_from_age(ret_df)

    return ret_df

### Race

In [27]:
## Create 'race' column using ethnicolr. 
def run_race(df):
    def first_name(name):
        m = re.match(r'(?:[A-Za-z]{0,2}\. )?([A-Za-z]+) ', name)
        if m:
            return m.group(1).title().strip()
        return ""

    temp_df = df.copy()
    temp_df['name'] = temp_df['name'].apply(lambda string: string.replace(' Sr.', ''))
    temp_df['name'] = temp_df['name'].apply(lambda string: string.replace(' Jr.', ''))
    temp_df['last_name'] = temp_df['name'].apply(lambda name: name.split(' ')[-1])
    # temp_df['last_name'] = temp_df['last_name'].apply(lambda name: name.lower())
    temp_df['first_name'] = temp_df['name'].apply(first_name)
    
    ## Highest probability race returned from Wikipedia data (using both first and last name)
    # wiki_pct = pred_wiki_name(temp_df, 'last_name', 'first_name').set_index('rowindex')  
    # df['race_wiki'] = wiki_pct['race']

    ## All probabilities from Census data (using last name only)
    pct = census_ln(temp_df, 'last_name').set_index(temp_df.index)[['pctwhite', 'pctblack', 'pctapi', 'pctaian', 'pct2prace', 'pcthispanic']]
    pct = pct.replace('(S)', np.nan).apply(pd.to_numeric).fillna(0)
    df['pred_race'] = pct.idxmax(axis = 1) 
    df['race_pred_value'] = pct.max(axis = 1)
    df.loc[df['race_pred_value'] == 0, 'pred_race'] = ''
    df['race_diff_highest'] = df['race_pred_value'] - pct.apply(lambda x: x.nlargest(2).iloc[1], axis = 1)


    return df

## Run functions

In [29]:
ret_df = df.copy()

ret_df = get_all_dates_RE(ret_df)
# ret_df['age_nlp'] = ret_df['para'].apply(lambda x: get_age_nlp(x))
# ret_df['age'] = ret_df['age'].fillna(ret_df['age_nlp'])
ret_df = get_all_dates_SU(ret_df)
ret_df['all_dates'] = ret_df['all_dates_su'] + ret_df['all_dates_re']

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Registering annotator sutime with class edu.stanford.nlp.time.TimeAnnotator
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words-distsim.tagger ... done [0.7 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.sequences.SeqClassifierFlags - sutime.includeRange=true
[main] INFO edu.stanford.nlp.sequences.SeqClassifierFlags - Unknown property: |sutime.includeRange|
[main] INFO edu.stanford.nlp.sequences.SeqClassifierFlags - sutime.language=english
[main] INFO edu.stanford.nlp.sequ

In [30]:
out_df = ret_df.copy()

out_df = parse_datestrings(out_df, YEAR, 'sutime')
out_df = add_scraped_dates(out_df)
out_df = birthday_from_age(out_df)

/home/ajing/.local/lib/python3.10/site-packages/dateparser/freshness_date_parser.py:76: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  now = self.get_local_tz().localize(now)
/home/ajing/.local/lib/python3.10/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/home/ajing/.local/lib/python3.10/site-packages/dateparser/freshness_date_parser.py:76: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migra

In [31]:
out_df = guess_gender(out_df)
out_df = simple_location_finder(out_df)
out_df = run_race(out_df)

In [32]:
out_df

,location,birthdate,deathdate,pubdate,url,funeral_home_name,funeral_home_address,para,site_published,name,...,dday_method,bday_string,bday_method,first_name,sex,funeral_home_ind,newspaper_ind,pred_race,race_pred_value,race_diff_highest
0,"Oxon Hill, MD",,,"Jan. 1, 2016",https://www.legacy.com/obituaries/washingtonpo...,,,"FELICITAS ARCENO CADISAL On Saturday, Decembe...",legacy,,...,sutime,NaN,NaN,,male,False,True,,0.00,0.00
1,"Washington, DC",,,"Jan. 1, 2016",https://www.legacy.com/obituaries/washingtonpo...,,,MARGARET LOUISE BUCHANAN (Yacura) Of Kenned...,legacy,,...,sutime,NaN,NaN,,male,False,True,,0.00,0.00
2,"Montgomery Village, MD",,,"Jan. 1, 2016",https://www.legacy.com/obituaries/washingtonpo...,,,"Charles Hastings Connolly On Tuesday, Decembe...",legacy,,...,sutime,NaN,NaN,,male,False,True,,0.00,0.00
3,"Washington, DC",1920-04-29,2015-12-17,,http://hosting-17924.tributes.com/obituary/sho...,,,"James was born on April 29, 1920 and passed aw...",tributes,James F. Carey,...,sutime,1920-04-29,sutime,James,male,False,False,pctwhite,83.21,70.46
4,"Washington, DC",1921,2015,"Jan. 1, 2016",https://www.legacy.com/obituaries/washingtonpo...,,,"EDWARD FRANCIS BARBANO, SR. Born on June 27, ...",legacy,,...,sutime,1921-06-27,sutime,,male,False,True,,0.00,0.00
5,"Washington, DC",,,"Jan. 1, 2016",https://www.legacy.com/obituaries/washingtonpo...,,,"EVELETH PEARL CAVALIERE Born January 14, 19...",legacy,,...,sutime,1921-01-14,sutime,,male,False,True,,0.00,0.00
6,"Washington, DC",,,"Jan. 2, 2016",https://www.legacy.com/obituaries/washingtonpo...,Robert O. Freeman Funeral Services,"1353 H Street, NE|Washington, DC","JEROME BENJAMIN CHASE On December 20, 2015....",legacy,,...,sutime,NaN,NaN,,male,True,True,,0.00,0.00
7,"Washington, DC",,,"Jan. 3, 2016",https://www.legacy.com/obituaries/washingtonpo...,Torchinsky Hebrew Funeral Home,"254 Carroll St. NW|Washington, DC","DR. LEWIS BATTIST On Wednesday, December 30, ...",legacy,,...,sutime,1929,sutime,,male,True,True,,0.00,0.00
8,"Silver Spring, MD",,,"Jan. 1, 2016",https://www.legacy.com/obituaries/washingtonpo...,Hines-Rinaldi Funeral Home,"11800 NEW HAMPSHIRE AVE|Silver Spring, MD","EDWARD C. CONINE, SR. On Sunday, December 27,...",legacy,,...,sutime,NaN,NaN,,male,True,True,,0.00,0.00
9,"Washington, DC",,,"Dec. 31, 2015",https://www.legacy.com/obituaries/washingtonpo...,,,"CALLAHAN DANIEL J. CALLAHAN, III Prominent Was...",legacy,,...,sutime,1950,sutime,,male,False,True,,0.00,0.00


In [12]:
sample_df = df.sample(10)

In [13]:
sample_df = hardcore_location_finder(sample_df)
display(sample_df)

,location,birthdate,deathdate,pubdate,url,para,site_published,name,funeral_home,all_locs
5741,Rheinland-Pfalz,,,"Jul. 12, 2020",,Janice Margaret Henderson Spotz (Age 80) Of...,legacy,JANICE SPOTZ,True,"[[None, Maryland], [None, Rheinland-Pfalz]]"
8862,Kentucky,,,"Nov. 29, 2020",,"CHARLOTTE DITTO HARBACH A beloved mother, gra...",legacy,CHARLOTTE HARBACH,True,"[[None, Kentucky], [None, Kentucky], [Houston,..."
5797,Florida,1933,2020,"Jul. 3, 2020",,"Rear Admiral Edward Anderson Wilkinson, Jr.,...",legacy,EDWARD WILKINSON Jr.,True,"[[None, Florida], [None, Alabama], [Hörselberg..."
4037,"Bethesda, Maryland",1937,2020,"May 24, 2020",,Ashby L. Chamberlin 1937 - 2020 Died Wednes...,legacy,ASHBY CHAMBERLIN,False,"[[None, Rhode Island], [South Norfolk, England..."
3003,"Washington, District of Columbia",,,"Apr. 19, 2020",,"DOUGLAS ROGER PORTER Douglas Roger Porter, 85...",legacy,DOUGLAS PORTER,True,"[[None, Virginia], [None, Virginia], [Lansing,..."
4276,Virginia,,,"Jun. 5, 2020",,Frances M. Kibler (Age 83) Passed away on May...,legacy,FRANCES KIBLER,True,"[[None, Virginia], [None, None], [None, Virgin..."
4387,"Chicago, Illinois",,,"May 17, 2020",,"Debra Ann Millenson (Age 72) Of Bethesda, ...",legacy,DEBRA Millenson,False,"[[Bethesda, Maryland], [None, Maryland], [New ..."
6269,"Bethesda, Maryland",,,"Jul. 24, 2020",,"ROBERT M. SANDLER ""BOB"" On July 21, 2020, Rob...",legacy,Robert Sandler,True,"[[Bethesda, Maryland], [None, Maryland], [Caga..."
95,"Zapopan, Jalisco",1950,2020,"Jan. 16, 2020",,OROZCO Dr. JOSE CARLOS OROZCO BUENROSTRO Leadi...,legacy,JOSE BUENROSTRO,False,"[[Zapopan, Jalisco], [None, None], [None, None..."
8613,"Washington, District of Columbia",,,"Nov. 1, 2020",,"ROBERT ELMON WEST (Age 91) Of Ft. Washington,...",legacy,ROBERT WEST,True,"[[Washington, District of Columbia], [None, Ma..."


In [14]:
simple_df = simple_location_finder(sample_df)
display(simple_df)
simple_df["all_locs"].iloc[8]

,location,birthdate,deathdate,pubdate,url,para,site_published,name,funeral_home,all_locs
5741,"Plata, MD",,,"Jul. 12, 2020",,Janice Margaret Henderson Spotz (Age 80) Of...,legacy,JANICE SPOTZ,True,"[[None, Maryland], [None, Rheinland-Pfalz]]"
8862,None,,,"Nov. 29, 2020",,"CHARLOTTE DITTO HARBACH A beloved mother, gra...",legacy,CHARLOTTE HARBACH,True,"[[None, Kentucky], [None, Kentucky], [Houston,..."
5797,"Lillian, AL",1933,2020,"Jul. 3, 2020",,"Rear Admiral Edward Anderson Wilkinson, Jr.,...",legacy,EDWARD WILKINSON Jr.,True,"[[None, Florida], [None, Alabama], [Hörselberg..."
4037,"Bethesda, MD",1937,2020,"May 24, 2020",,Ashby L. Chamberlin 1937 - 2020 Died Wednes...,legacy,ASHBY CHAMBERLIN,True,"[[None, Rhode Island], [South Norfolk, England..."
3003,None,,,"Apr. 19, 2020",,"DOUGLAS ROGER PORTER Douglas Roger Porter, 85...",legacy,DOUGLAS PORTER,True,"[[None, Virginia], [None, Virginia], [Lansing,..."
4276,"Burke, VA",,,"Jun. 5, 2020",,Frances M. Kibler (Age 83) Passed away on May...,legacy,FRANCES KIBLER,True,"[[None, Virginia], [None, None], [None, Virgin..."
4387,"Skokie, IL",,,"May 17, 2020",,"Debra Ann Millenson (Age 72) Of Bethesda, ...",legacy,DEBRA Millenson,True,"[[Bethesda, Maryland], [None, Maryland], [New ..."
6269,"Washington, DC",,,"Jul. 24, 2020",,"ROBERT M. SANDLER ""BOB"" On July 21, 2020, Rob...",legacy,Robert Sandler,True,"[[Bethesda, Maryland], [None, Maryland], [Caga..."
95,None,1950,2020,"Jan. 16, 2020",,OROZCO Dr. JOSE CARLOS OROZCO BUENROSTRO Leadi...,legacy,JOSE BUENROSTRO,True,"[[Zapopan, Jalisco], [None, None], [None, None..."
8613,"Oxon Hill, MD",,,"Nov. 1, 2020",,"ROBERT ELMON WEST (Age 91) Of Ft. Washington,...",legacy,ROBERT WEST,True,"[[Washington, District of Columbia], [None, Ma..."


[['Zapopan', 'Jalisco'],
 [None, None],
 [None, None],
 ['Ciudad de México', 'Ciudad de México'],
 [None, None],
 ['New York', 'New York'],
 [None, None],
 ['Guadalajara', 'Jalisco'],
 ['Guadalajara', 'Jalisco']]

In [15]:
df = pd.read_csv("test-guess_gender-2020.csv")
df

,index,location,birthdate,deathdate,pubdate,url,para,site_published,name,first_name,sex
0,0,"Silver Spring, MD",NaN,NaN,"Jan. 10, 2020",NaN,"Stanley J. Abremski On Sunday January 5, 2020...",legacy,STANLEY ABREMSKI,Stanley,male
1,1,NaN,1921,2019,"Jan. 11, 2020",NaN,ACKERMAN Helen Johnson Ackerman Helen Ackerman...,legacy,HELEN ACKERMAN,Helen,female
2,2,NaN,NaN,NaN,"Jan. 8, 2020",NaN,"HERBERT LOUIS ACORS, SR. Entered into eternal...",legacy,HERBERT ACORS Sr.,Herbert,male
3,3,NaN,NaN,NaN,"Jan. 5, 2020",NaN,"James Agenbroad Born Dayton, Ohio on December...",legacy,JAMES AGENBROAD,James,male
4,4,"Fairfax, VA",1930,2020,"Jan. 10, 2020",NaN,"Aguanno, Edwin Sept. 8 1930- Jan 1 2020 Edwin ...",legacy,Edwin Aguanno,Edwin,male
...,...,...,...,...,...,...,...,...,...,...,...
9398,9843,NaN,1944,2020,"Dec. 10, 2020",NaN,"James Bryson Worsham James Bryson Worsham, 76...",legacy,JAMES WORSHAM,James,male
9399,9844,"Fairfax, VA",1938,2020,"Dec. 29, 2020",NaN,"Arthur Lee Wright April 10, 1938 - December 1...",legacy,ARTHUR WRIGHT,Arthur,male
9400,9845,"Camp Springs, MD",1920,2020,"Dec. 26, 2020",NaN,YOUNG CREDELLA DERRICOTTE YOUNG Credella Derri...,legacy,Credella Young,Credella,male
9401,9846,"Suitland, MD",NaN,NaN,"Dec. 18, 2020",NaN,FRED JEROME YOUNG Entered into eternal rest o...,legacy,FRED YOUNG,Fred,male


### Obtaining proportion FH 

In [16]:
# month_dict = {'01': 'Jan', '02':'Feb', '03':'Mar', '04':'Apr', '05':'May', '06':'Jun', '07':'Jul', '08':'Aug',
# '09':'Sep', '10':'Oct', '11':'Nov', '12':'Dec'}

# ## Populating pubdate for non-legacy websites using death date
# for i in df.index:
#     if (df.loc[i, 'pubdate'] == ''):
#         ## Tributes.com deathdate present
#         if (df.loc[i, 'deathdate'] != ''):
#             death_month = df.loc[i, 'deathdate'][5:7]
#             df.loc[i, 'pubdate'] = month_dict[death_month]
#         else:
#             # deathdate not present, find in paragraph
#             pubdate_finder = re.compile(r'passed away .+ (Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)')
#             found = re.search(pubdate_finder, df.loc[i, 'para'])
#             found = '' if found is None else found[0]
#             finder2 = re.compile(r'(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)')
#             death_month = re.search(finder2, found)
#             df.loc[i, 'pubdate'] = '' if death_month is None else death_month[0]

# df_prop = pd.DataFrame(columns = ['Proportion Funeral Homes {}'.format(YEAR)])

# for m in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
#     month_data = df[df['pubdate'].str[:3] == m]
#     frac = month_data['location'].apply(lambda x: bool(x)).sum()/len(month_data)
#     df_prop.loc[len(df_prop)] = frac
        
# df_prop.to_csv('FH_frac.csv', mode = 'a')

## Visualization

In [17]:
dateframe = wash_obits[wash_obits["birthday"].notnull()]
dateframe["birth_month"] = dateframe["birthday"].dt.month
dateframe["birth_year"] = dateframe["birthday"].dt.year
dateframe["age"] = (dateframe["deathday"] - dateframe["birthday"]).dt.total_seconds()/3600/24/365
youngs = dateframe[dateframe["age"] < 30]
# display(youngs)
# display(youngs["para"])
dateframe = dateframe.groupby(["birth_month", "birth_year"])["age"].mean()
dateframe = pd.DataFrame(dateframe)
date_pivot = dateframe.pivot_table(index="birth_month", columns="birth_year", values="age")
#dateframe
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(date_pivot)

NameError: name 'wash_obits' is not defined

In [ ]:
sns.distplot(dateframe["age"])